# Création de nouveaux sous-titres segmentés en phrase

Ce notebook permet de créer de nouveaux fichiers de sous-titres à l'aide de deux fichiers : 
1. Un fichier contenant les phrases (sous titres)
2. Un fichier de sous-titres, sous segmenté pour que chaque sous-titre correspondent à une phrase ou à un morceau de phrase (et non deux phrases dans un même sous-titre)

In [2]:
import os
import re
import module_traitement as m 

In [162]:
# Pour Matignon - LSF 
pre_seg_file = "../data/new_segmentation_cr/"
sentence_file = "../data/sentence_matignon/"
resultat = m.lister_fichiers_with_path(pre_seg_file)
path_sentence = m.lister_fichiers_with_path(sentence_file)
resultat_output = m.lister_fichiers(pre_seg_file)
sentences_only = m.lister_fichiers(sentence_file)
output_folder = "../data/new_cr_sub/"

In [70]:
# Pour Mediapi : 
pre_seg_file = "../data/new_segmentation_mediapi/"
sentence_file = "../data/sentence_mediapi/"
resultat = m.lister_fichiers_with_path(pre_seg_file)
path_sentence = m.lister_fichiers_with_path(sentence_file)
resultat_output = m.lister_fichiers(pre_seg_file)
sentences_only = m.lister_fichiers(sentence_file)
output_folder = "../data/new_mediapi_sub/"

In [85]:
# TEST Pour Matignon - LSF 
pre_seg_file = "test_new_seg_cr/"
sentence_file = "try/"
resultat = m.lister_fichiers_with_path(pre_seg_file)
path_sentence = m.lister_fichiers_with_path(sentence_file)
resultat_output = m.lister_fichiers(pre_seg_file)
sentences_only = m.lister_fichiers(sentence_file)
output_folder = "test_sent_seg_cr/"

In [16]:
# TEST Mediapi
pre_seg_file = "test_new_seg_mediapi/"
sentence_file = "test_clean_sent_mediapi/"
resultat = m.lister_fichiers_with_path(pre_seg_file)
path_sentence = m.lister_fichiers_with_path(sentence_file)
resultat_output = m.lister_fichiers(pre_seg_file)
sentences_only = m.lister_fichiers(sentence_file)
output_folder = "test_sent_seg_mediapi/"

In [4]:
import module_traitement as m 

In [86]:
# Création de nouveaux fichiers de sous-titres
for sub,sub_name,phr,phr_name in zip(resultat,resultat_output,path_sentence,sentences_only):
    print(f"TRAITEMENT : {sub_name} && {phr_name}")
    print(f"TRAITEMENT : {sub} && {phr}")
    dict_sub = m.get_dict_vtt(sub)
    sentences = m.get_sentences(phr)
    keys = list(dict_sub.keys())
    i = 0
    j = 0
    mm = 0
    new_dict = {}
    content = ""
    while i < len(keys) and j < len(sentences):
        #print(mm)
        key = keys[i]
        value = dict_sub[key]
        unite = value['text'].strip()
        sent = sentences[j].strip()
        #print(f"'{unite} --> '{sent}'")
        # Je regarde si l'unité est dans la phrase. Si elle est dans la phrase, j'ajoute l'information start, et tant que les unités sont dans la même phrase je concatène pour ajouter l'information du texte
        if unite in sent:
            #print(f"'{unite} --> '{sent}'")
            if mm not in new_dict:
                new_dict[mm] = {"start": value['start']}
            if 'text' not in new_dict[mm]:
                new_dict[mm]["text"] = unite.strip()
            else:
                new_dict[mm]['text'] += f" {unite.strip()}"
            i = i +1

            if i == len(keys):
                new_dict[mm]['end'] = value['end']
        else:
            old_key = keys[i - 1]
            if mm in new_dict and 'end' not in new_dict[mm]:
                new_dict[mm]['end'] = dict_sub[old_key]['end']
                #print(new_dict)
                mm += 1
            j = j +1
    m.create_vtt_file(new_dict,f"{output_folder}/{sub_name}")

TRAITEMENT : 8ZUIw7jcaZE.vtt && 8ZUIw7jcaZE.vtt
TRAITEMENT : test_new_seg_cr/8ZUIw7jcaZE.vtt && try/8ZUIw7jcaZE.vtt
TRAITEMENT : hegyfM0YipI.vtt && hegyfM0YipI.vtt
TRAITEMENT : test_new_seg_cr/hegyfM0YipI.vtt && try/hegyfM0YipI.vtt
TRAITEMENT : 8IqhpOiPMxY.vtt && 8IqhpOiPMxY.vtt
TRAITEMENT : test_new_seg_cr/8IqhpOiPMxY.vtt && try/8IqhpOiPMxY.vtt
TRAITEMENT : H448NJiwMRI.vtt && H448NJiwMRI.vtt
TRAITEMENT : test_new_seg_cr/H448NJiwMRI.vtt && try/H448NJiwMRI.vtt
TRAITEMENT : jNouEg1YgKQ.vtt && jNouEg1YgKQ.vtt
TRAITEMENT : test_new_seg_cr/jNouEg1YgKQ.vtt && try/jNouEg1YgKQ.vtt
TRAITEMENT : 6fsIXStr6w4.vtt && 6fsIXStr6w4.vtt
TRAITEMENT : test_new_seg_cr/6fsIXStr6w4.vtt && try/6fsIXStr6w4.vtt
TRAITEMENT : CDsP8gaVGbg.vtt && CDsP8gaVGbg.vtt
TRAITEMENT : test_new_seg_cr/CDsP8gaVGbg.vtt && try/CDsP8gaVGbg.vtt
TRAITEMENT : rpcc3B2P31Q.vtt && rpcc3B2P31Q.vtt
TRAITEMENT : test_new_seg_cr/rpcc3B2P31Q.vtt && try/rpcc3B2P31Q.vtt
TRAITEMENT : K7WqKupeGVk.vtt && K7WqKupeGVk.vtt
TRAITEMENT : test_new_se

## Vérifications manuelles

On compare le fichier de phrases avec les nouveaux sous-titres, pour vérifier si on a la même chose. 

S'il y a une différence, on note le fichier et l'endroit, ce qui permet de faire une correction.

On corriger : 
- soit le fichier de phrases (si la phrases a été mal découpé par le modèle de langue)
- soit le fichier sur-segmenté, s'il y a eu un problème de segmentation

In [87]:
path_sentence = m.lister_fichiers_with_path("try/") # vérifier qu'on charge le bon fichier !
output_folder = m.lister_fichiers_with_path("test_sent_seg_cr")
print(output_folder)

['test_sent_seg_cr/8ZUIw7jcaZE.vtt', 'test_sent_seg_cr/hegyfM0YipI.vtt', 'test_sent_seg_cr/8IqhpOiPMxY.vtt', 'test_sent_seg_cr/H448NJiwMRI.vtt', 'test_sent_seg_cr/jNouEg1YgKQ.vtt', 'test_sent_seg_cr/6fsIXStr6w4.vtt', 'test_sent_seg_cr/CDsP8gaVGbg.vtt', 'test_sent_seg_cr/rpcc3B2P31Q.vtt', 'test_sent_seg_cr/K7WqKupeGVk.vtt', 'test_sent_seg_cr/nK47Jj-GdTo.vtt', 'test_sent_seg_cr/YmXGfLxTvFQ.vtt', 'test_sent_seg_cr/F5-w4cvC_L0.vtt', 'test_sent_seg_cr/-LhfYZ1ihpI.vtt', 'test_sent_seg_cr/dGxmroECIlU.vtt', 'test_sent_seg_cr/jo4puWxt-Ik.vtt', 'test_sent_seg_cr/VZeIXQAt2C8.vtt', 'test_sent_seg_cr/aKGSwBZpRXw.vtt', 'test_sent_seg_cr/qVYiLFZojws.vtt', 'test_sent_seg_cr/j6bYWLKPuPs.vtt', 'test_sent_seg_cr/qUZ5TOoL1EE.vtt', 'test_sent_seg_cr/V-XRefvjCEU.vtt', 'test_sent_seg_cr/aTtX70EXGVo.vtt', 'test_sent_seg_cr/pyC6EXZ449c.vtt', 'test_sent_seg_cr/CxgdUjywiDE.vtt', 'test_sent_seg_cr/awTzYOLwLSc.vtt', 'test_sent_seg_cr/1MHphyCtLLE.vtt', 'test_sent_seg_cr/Y0AmWgJwbVg.vtt', 'test_sent_seg_cr/3TEX9ruha

In [88]:
path_sentence

['try/8ZUIw7jcaZE.vtt',
 'try/hegyfM0YipI.vtt',
 'try/8IqhpOiPMxY.vtt',
 'try/H448NJiwMRI.vtt',
 'try/jNouEg1YgKQ.vtt',
 'try/6fsIXStr6w4.vtt',
 'try/CDsP8gaVGbg.vtt',
 'try/rpcc3B2P31Q.vtt',
 'try/K7WqKupeGVk.vtt',
 'try/nK47Jj-GdTo.vtt',
 'try/YmXGfLxTvFQ.vtt',
 'try/F5-w4cvC_L0.vtt',
 'try/-LhfYZ1ihpI.vtt',
 'try/dGxmroECIlU.vtt',
 'try/jo4puWxt-Ik.vtt',
 'try/VZeIXQAt2C8.vtt',
 'try/aKGSwBZpRXw.vtt',
 'try/qVYiLFZojws.vtt',
 'try/j6bYWLKPuPs.vtt',
 'try/qUZ5TOoL1EE.vtt',
 'try/V-XRefvjCEU.vtt',
 'try/aTtX70EXGVo.vtt',
 'try/pyC6EXZ449c.vtt',
 'try/CxgdUjywiDE.vtt',
 'try/awTzYOLwLSc.vtt',
 'try/1MHphyCtLLE.vtt',
 'try/Y0AmWgJwbVg.vtt',
 'try/3TEX9ruhaXo.vtt',
 'try/1ILfD_BjLNk.vtt',
 'try/vlNNOM4i3Q0.vtt',
 'try/M5SVtrVMQm0.vtt',
 'try/wMDNQG0oA7k.vtt',
 'try/LicjiPStTmU.vtt',
 'try/zwr3T94t0ZE.vtt',
 'try/_Lx00No3bZc.vtt',
 'try/B62_uSapEhI.vtt',
 'try/1AjRdJ5d_Ww.vtt',
 'try/ub_iGqVn8EU.vtt',
 'try/Yw0atlzQbEs.vtt',
 'try/KhxgwQDJpCg.vtt',
 'try/RenNdZa3-QY.vtt',
 'try/X3SSQD7qsT

In [89]:
nb_file = 0
name_file = []
good_file = []
for sub_file, sent_file in zip(output_folder,path_sentence):
    sub_dict = m.get_dict_vtt(sub_file)
    sentence = m.get_sentences(sent_file)
    new_sent = []
    for k,v in sub_dict.items():
        for kk,vv in v.items():
            if kk == "text":
                new_sent.append(vv)
    differences = m.comparer_listes(new_sent, sentence)
    if differences:
        print(f"TRAITEMENT : {sub_file} && {sent_file}")
        for position, element_a, element_b in differences:
            nb_file = nb_file +1
            print(f"Différence à la position {position}: {element_a} vs {element_b}")
            name_file.append(sub_file)
            break
    else:
        good_file.append(sub_file)

print(f"Nombres de fichiers contenant une erreur : {nb_file}")

test_sent_seg_cr/8ZUIw7jcaZE.vtt try/8ZUIw7jcaZE.vtt
test_sent_seg_cr/hegyfM0YipI.vtt try/hegyfM0YipI.vtt
test_sent_seg_cr/8IqhpOiPMxY.vtt try/8IqhpOiPMxY.vtt
TRAITEMENT : test_sent_seg_cr/8IqhpOiPMxY.vtt && try/8IqhpOiPMxY.vtt
Différence à la position 367: Bonjour Monsieur le Ministre Luc (? ), pour TF1, LCI. vs Bonjour Monsieur le Ministre Luc (?), pour TF1, LCI.
test_sent_seg_cr/H448NJiwMRI.vtt try/H448NJiwMRI.vtt
test_sent_seg_cr/jNouEg1YgKQ.vtt try/jNouEg1YgKQ.vtt
TRAITEMENT : test_sent_seg_cr/jNouEg1YgKQ.vtt && try/jNouEg1YgKQ.vtt
Différence à la position 75: Un gouvernement qui est en mouvement aussi parce que, je vous l'ai dit, il est renouvelé au même titre que sa méthode. Un gouvernement qui est en mouvement vs Un gouvernement qui est en mouvement aussi parce que, je vous l'ai dit, il est renouvelé au même titre que sa méthode.
test_sent_seg_cr/6fsIXStr6w4.vtt try/6fsIXStr6w4.vtt
TRAITEMENT : test_sent_seg_cr/6fsIXStr6w4.vtt && try/6fsIXStr6w4.vtt
Différence à la position 150

In [73]:
len(name_file)

22

In [15]:
# Si besoin de déplacer des fichiers 

# Liste des chemins des fichiers à déplacer
chemins_des_fichiers = good_file

# Dossier de destination
dossier_de_destination = "gold_sent_seg_mediapi/"

# Appel de la fonction de déplacement des fichiers
m.deplacer_fichiers(chemins_des_fichiers, dossier_de_destination)


Le fichier 'b7f2d8f0c3.vtt' a été déplacé vers 'gold_sent_seg_mediapi/'.
Le fichier '44f554f914.vtt' a été déplacé vers 'gold_sent_seg_mediapi/'.
Le fichier 'b9a51f4361.vtt' a été déplacé vers 'gold_sent_seg_mediapi/'.
Le fichier '4e073949b1.vtt' a été déplacé vers 'gold_sent_seg_mediapi/'.
Le fichier '0b1437bc85.vtt' a été déplacé vers 'gold_sent_seg_mediapi/'.
Le fichier '752500b761.vtt' a été déplacé vers 'gold_sent_seg_mediapi/'.
Le fichier 'cba6cefad2.vtt' a été déplacé vers 'gold_sent_seg_mediapi/'.
Le fichier '41c606553f.vtt' a été déplacé vers 'gold_sent_seg_mediapi/'.
Le fichier '3f1b2118ca.vtt' a été déplacé vers 'gold_sent_seg_mediapi/'.
Le fichier 'ed969c3e70.vtt' a été déplacé vers 'gold_sent_seg_mediapi/'.
Le fichier '26f1ff8385.vtt' a été déplacé vers 'gold_sent_seg_mediapi/'.
Le fichier '15abfc95ae.vtt' a été déplacé vers 'gold_sent_seg_mediapi/'.
Le fichier '495145911e.vtt' a été déplacé vers 'gold_sent_seg_mediapi/'.
Le fichier '0fd91cb814.vtt' a été déplacé vers 'gol